In [1]:
import os
import pandas as pd
import numpy as np
import glob
from datasets import Dataset
from transformers import AutoTokenizer

/home/yujia/miniconda3/envs/incivility/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-04-11 01:47:24.997068: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-11 01:47:25.888828: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-04-11 01:47:25.888927: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer

In [3]:
dir = './data/comments'
datasets_dir = "./data/datasets"

files = sorted(glob.glob(f'{dir}/*.pickle'))

print(files)

['./data/comments/20200201_20200315_all_comments_with_labels.pickle', './data/comments/20200201_20200315_all_comments_with_labels_dup.pickle', './data/comments/20200315_20200528_all_comments_with_labels.pickle', './data/comments/20200528_20200713_all_comments_with_labels.pickle', './data/comments/20200713_20200813_all_comments_roberta.pickle', './data/comments/20200713_20200813_all_comments_with_labels.pickle', './data/comments/20200813_20201025_all_comments_with_labels.pickle', './data/comments/20201025_20201128_all_comments_with_labels.pickle', './data/comments/20201129_20210201_all_comments_with_labels.pickle']


In [5]:
# for f in files[2:3]:
f = files[2]
head, tail = os.path.split(f)
fname = tail.split('.')[0]
print(fname)

20200315_20200528_all_comments_with_labels


In [4]:
df = pd.read_pickle(f)
print(len(df))
df = df.dropna(subset=['body', 'id'])
df = df[(df['author'] != '[deleted]') & (df['author'] != '[removed]') & (df['body'] != '[deleted]') & (df['body'] != '[removed]')]
print(len(df))


6310154
6310154


In [10]:
df = df.iloc[:3000000,:]
dataset = Dataset.from_pandas(df[['id', 'body']])
del df

In [ ]:
dataset

In [6]:
dataset_path = os.path.join(datasets_dir, fname+"_first_roberta")
dataset = Dataset.load_from_disk(dataset_path)

In [11]:
tokenizer = AutoTokenizer.from_pretrained("roberta-base")
dataset = dataset.map(lambda e: tokenizer(e['body'], truncation=True, padding='max_length'), batched=True)
dataset.save_to_disk(dataset_path)


100%|██████████| 3311/3311 [08:35<00:00,  6.43ba/s]


In [ ]:
# from resource import getrusage, RUSAGE_SELF
# print(getrusage(RUSAGE_SELF).ru_maxrss)